In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1")
         .getOrCreate())
sc = spark.sparkContext

22/01/27 12:10:10 WARN Utils: Your hostname, stepwise resolves to a loopback address: 127.0.1.1; using 192.168.100.32 instead (on interface wlp3s0)
22/01/27 12:10:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zalda/Desktop/Projects/Learn%20Data%20Engineering/document_streaming_project/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zalda/.ivy2/cache
The jars for the packages stored in: /home/zalda/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aed8c126-288e-4569-953b-2c537b294eeb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.1 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	

In [2]:
# # Read the message from the kafka stream
# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribe", "ingestion-topic") \
#   .load()
# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Subscribe to 1 topic
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "ingestion-topic").load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [3]:
#Create a small temporary view for SparkSQL
df.createOrReplaceTempView("message")

In [4]:
# Write out the message to the console of the environment
res = spark.sql("SELECT * from message")
res.writeStream.format("console").outputMode("append").start() 

22/01/27 12:10:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5d919f79-9bc3-4290-a62e-db9e85cfdd1b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/01/27 12:10:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [5]:
# Write the message back into Kafka in another topic that you are going to listen to with a local consumer
ds = df.writeStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("topic", "spark-output").option("checkpointLocation", "/tmp").start().awaitTermination()

22/01/27 12:10:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|     5|2022-01-27 12:11:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|     6|2022-01-27 12:11:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|     7|2022-01-27 12:16:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+



...

KeyboardInterrupt: 

22/01/27 14:10:13 ERROR MicroBatchExecution: Query [id = 28650e55-f70f-4bf5-a104-59bb3e824f04, runId = b8b270a7-6db0-437e-85c2-d1901a82894b] terminated with error
java.lang.AssertionError: assertion failed: Concurrent update to the log. Multiple streaming jobs detected for 5
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.$anonfun$constructNextBatch$14(MicroBatchExecution.scala:443)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.sql.execution.streaming.ProgressReporter.reportTimeTaken(ProgressReporter.scala:375)
	at org.apache.spark.sql.execution.streaming.ProgressReporter.reportTimeTaken$(ProgressReporter.scala:373)
	at org.apache.spark.sql.execution.streaming.StreamExecution.reportTimeTaken(StreamExecution.scala:69)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.$anonfun$constructNextBatch$1(MicroBatchExecution.scala:440)
	at scala.runtime.java8.JFunction0$mc

-------------------------------------------
Batch: 4
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|     8|2022-01-27 14:10:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-

-------------------------------------------
Batch: 6
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|    10|2022-01-27 14:31:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 7
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-

-------------------------------------------
Batch: 8
-------------------------------------------
+----+--------------------+---------------+---------+------+--------------------+-------------+
| key|               value|          topic|partition|offset|           timestamp|timestampType|
+----+--------------------+---------------+---------+------+--------------------+-------------+
|null|[7B 22 49 6E 76 6...|ingestion-topic|        0|    12|2022-01-27 15:50:...|            0|
+----+--------------------+---------------+---------+------+--------------------+-------------+

